# Run the CICI SAFE vSDX Using Chameleon

#### Configure the Envrionment

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-819993'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/.ssh/id_rsa

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
# NODE_RESERVATION_ID must be for Skylake nodes
export SDX_RESERVATION_ID='c3f9c003-888b-4dbd-86c6-e79a388d136a'
export CLIENT_RESERVATION_ID='c3f9c003-888b-4dbd-86c6-e79a388d136a'
export RIAK_RESERVATION_ID='c3f9c003-888b-4dbd-86c6-e79a388d136a'

#Set the working directory
export WORKING_DIR=${HOME}/work/SDX

#### Start RIAK Server

In [ ]:
cd $WORKING_DIR

RIAK_NETWORK='sharednet1'
RIAK_NODE_NAME=${USERNAME}-riak
RIAK_STACK_NAME=${USERNAME}-riak

In [ ]:
echo Creating RIAK server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template riak.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${RIAK_RESERVATION_ID}" \
        --parameter "network_name=${RIAK_NETWORK}" \
        --parameter "riak_node_name=${RIAK_NODE_NAME}" \
        --wait \
        ${RIAK_STACK_NAME}

In [ ]:
echo RIAK server creation complete! 
echo ${RIAK_NODE_NAME} status `openstack server show  --format value -c status ${RIAK_NODE_NAME}`

RIAK_PUBLIC_IP=`openstack server show  --format value -c addresses ${RIAK_NODE_NAME} | cut -d " " -f 2`
echo The RIAK se public IP is $RIAK_PUBLIC_IP

#### Start SDX Server

In [ ]:
cd $WORKING_DIR

SDX_NETWORK='sharednet1'
SDX_NODE_NAME=${USERNAME}-sdx
SDX_STACK_NAME=${USERNAME}-sdx

In [ ]:
echo Creating SDX server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template sdx.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${SDX_RESERVATION_ID}" \
        --parameter "network_name=${SDX_NETWORK}" \
        --parameter "sdx_node_name=${SDX_NODE_NAME}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --wait \
        ${SDX_STACK_NAME}

In [ ]:
echo SDX server creation complete! 
echo ${SDX_NODE_NAME} status `openstack server show  --format value -c status ${SDX_NODE_NAME}`

SDX_PUBLIC_IP=`openstack server show  --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo The SDX se public IP is $SDX_PUBLIC_IP

In [ ]:
SDX_PUBLIC_IP=`openstack server show --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo SDX_PUBLIC_IP is $SDX_PUBLIC_IP

#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo SDX node ready for ssh

In [ ]:
#Tail log until complete. Note: YOU MUST MANUALLY STOP THIS TO CONTINUE!
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo tail /root/boot.log

#### Configure the SDX node

In [ ]:
#Copy GENI pem file to node
REMOTE_GENI_CERT=/home/cc/geni.pem
GENI_CERT=${HOME}/geni.pem

echo Copying GENI cert to cc@$SDX_PUBLIC_IP:~/geni.pem
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    ${GENI_CERT} \
    cc@${SDX_PUBLIC_IP}:${REMOTE_GENI_CERT}
echo Done

In [ ]:
LOCAL_SDX_CONFIG=${WORKING_DIR}/sdx.conf

REMOTE_SDX_CONFIG=/home/cc/sdx.conf
REMOTE_WORKING_DIR=/root
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

DATAPLANE_SLICE_NAME=${USERNAME}-sdx-dataplane

cat << EOF > $LOCAL_SDX_CONFIG
config {
safe=false
slicename="${DATAPLANE_SLICE_NAME}"
type="server"
sshkey="/root/.ssh/id_rsa"
safekey="sdx"
exogenipem="${REMOTE_GENI_CERT}"
exogenism="https://geni.renci.org:11443/orca/xmlrpc"
ipprefix="192.168.128.1/24"
riakserver="${RIAK_PUBLIC_IP}"
serverinslice=false
safeserver="${SDX_PUBLIC_IP}"
plexusserver="${SDX_PUBLIC_IP}"
serverurl="http://0.0.0.0:8888/"
scriptsdir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/scripts/"
clientsites:"UFL:UNF"
controllersite:"UFL (Gainesville, FL USA) XO Rack"
serversite:"RENCI (Chapel Hill, NC USA) XO Rack"
}
EOF

echo Copying SDX config to cc@$SDX_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_SDX_CONFIG \
    cc@${SDX_PUBLIC_IP}:${REMOTE_SDX_CONFIG}
echo Done

#### Start the Dataplane Slice

In [ ]:
echo Start SDX dataplane Slice
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo ${REMOTE_BIN_DIR}/SafeSdxSliceServer -c $REMOTE_SDX_CONFIG  
echo Slice Creation Done!

#### Start the SDX Server

In [ ]:
echo Start the SDX Server
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    "sudo pkill java; sudo ${REMOTE_BIN_DIR}/SafeSdxServer -c $REMOTE_SDX_CONFIG &"

# Please Cleanup Your Resources!!!

#### Delete Dataplane Slice

In [ ]:
#need to install exogeni phython script in the container
#python delete ${DATAPLANE_SLICE_NAME}

echo Delete SDX Slice

echo $REMOTE_SDX_CONFIG

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxSliceServer -c $REMOTE_SDX_CONFIG -d"
    
echo Slice Deletion Done

#### Delete SDX Stack

In [ ]:
echo Deleting stack ${SDX_STACK_NAME}
openstack stack delete -y --wait ${SDX_STACK_NAME}
echo Stack ${SDX_STACK_NAME} deleted

#### Delete RIAK Stack

In [ ]:
echo Deleting stack ${RIAK_STACK_NAME}
openstack stack delete -y --wait ${RIAK_STACK_NAME}
echo Stack ${RIAK_STACK_NAME} deleted